## Q&A Agent
#### General Idea: 
I investigate the dataset and found that some of the transactions can up to hundred thousands. At the very begining, I'm thinking to implement a RAG and store the data into a vector data store. But unfortunately, hundred thousands of transactions would overwhelm the LLM maximum context length. 

After much brainstorming, the huge amount of data can only be handled in SQL before fetching to LLM. Hence, the general idea of the LLM implementation would be focusing on how to generate an accurate SQL query to answer the user question.

The prompt engineering part would be using the combination of chain of thought and showing some examples. There are totally three prompts implemented in this project:
1. structured_filter_prompt: To investigate if there is any area we need to apply WHERE syntax. 
2. init_sql_prompt: To generate SQL query to answer the questions.
3. synthesizer prompt: To synthesize results and questions for a human understandable response.

Groq - LLama 3 70b and gemini-pro are used in this project. After several testing, LLama 3 70b showed a better output of SQL query as compared to other model including CodeQwen 1.5 7b. 

Main implementation idea:
Make sure all the important outputs are structured output so that we can consume the output in programamtical way.

Future Improvements:
- Introduce Agent into the LLM so that different tasks can be solve by different functions.
- Fine tuning a small LM for specific tasks.


##### Key in your client id and question here: (Actual implementation will have client id captured automatically when client ask question from their accounts)

In [1]:
client_id = 6
client_question = "What is the amount I have spent on shopping in the last 10 months?"

## Import Main LLM and Smart LLM

In [2]:
from genai.llms import groq_llm, main_llm

smart_llm = groq_llm()
llm = main_llm()

## Data Exploration and Preprocessing

In [3]:
import pandas as pd

## Load data into pandas dataframe
df = pd.read_csv("./data.csv")

In [4]:
## Check column names
df.columns

Index(['clnt_id', 'bank_id', 'acc_id', 'txn_id', 'txn_date', 'desc', 'amt',
       'cat', 'merchant'],
      dtype='object')

In [5]:
## Check if there are any null values in the dataset
df.isnull().sum()

clnt_id          0
bank_id          0
acc_id           0
txn_id           0
txn_date         0
desc             0
amt              0
cat              0
merchant    124881
dtype: int64

In [6]:
from data_preprocessing import DataPreprocessing
data_prep = DataPreprocessing()

df = data_prep.column_rename(df)
df = data_prep.null_replace(df)
df = data_prep.datetime_format(df)

## Extract Filter object from query for SQL generation

In [7]:
## Concatenate the client context and question.
client_id_context = f"My client_id is {client_id}. "
query = client_id_context + client_question

In [8]:
## Filter only relevant data for the client to make sure client cannot access other data
df = df[df['client_id']==client_id]

In [9]:
## Generate a list of unique categories from the given DataFrame.
category_list = df['category'].unique().tolist()

## Generate a list of unique merchants from the given dataset.
merchant_list = df['merchant'].unique().tolist()

In [10]:
from genai.chain import extract_structured_filter
from genai.output_parser import structured_output_parser

## Define outpur parser
output_parser_llm = structured_output_parser()

## Extract filter objects for SQL generation
structured_object = extract_structured_filter(output_parser_llm, query, category_list, merchant_list, llm)

## Validate if the filter object is a valid category or merchant

In [11]:
## Validate filter object for SQL generation
## If the result is False, we need to rerun the structured_object query again - meaning that the LLM generate its own items will cause empty result coming back
def valid_filter_object(structured_object, object_list, object):
    is_match = True
    if len(structured_object[object]) > 0:
        for item in structured_object[object]:
            if item not in object_list:
                is_match = False
    return is_match

is_cat_match = valid_filter_object(structured_object, category_list, 'category')
is_merc_match = valid_filter_object(structured_object, merchant_list, 'merchant')

is_match = is_cat_match and is_merc_match
print(is_match)

True


In [12]:
## Filter a dictionary to remove empty values
## Prepare to include the filter object into the prompt for SQL generation
filtered_structured_object = {key: value for key, value in structured_object.items() if value}
filtered_structured_object

{'category': ['Shops']}

## SQL generation - Groq LLama 3 70b
#### LLama 3 70b has a better reasoning skills as compared to other models. Would like to use his reasonning skill to generate a SQL query.

In [13]:
from genai.prompts import init_sql_prompt

## Use Gemini Pro to generate sql query
sql_prompt = init_sql_prompt(filtered_structured_object, query)
valid_response = smart_llm.invoke(sql_prompt)
valid_sql = str(valid_response.content)
print(valid_sql)

Here is the generated DuckDB SQL query based on the provided details:

```
SELECT SUM(amount) 
FROM df 
WHERE client_id = 6 
AND category = 'Shops' 
AND transaction_date > '2023-06-25' 
AND transaction_date <= '2024-04-25';
```

This query filters the data based on the provided filter object, which includes the category 'Shops', and also applies the date range filter to get the transactions in the last 10 months.


## Extract SQL query from the response

In [14]:
## Response is not purely SQL, so further preprocessing is needed to extract SQL code from it.
sql = llm.invoke(f"Please extract only SQL code from the given response. Response: {valid_sql}")
sql = sql.replace("\n", " ").replace("sql", "").replace("`", "").replace("duckdb", "")
print(sql)

 SELECT SUM(amount)  FROM df  WHERE client_id = 6  AND category = 'Shops'  AND transaction_date > '2023-06-25'  AND transaction_date <= '2024-04-25'; 


## Make use of DuckDB to use SQL and obtain results as Pandas DataFrame

In [15]:
import duckdb

results = duckdb.sql(sql).df()
results

,sum(amount)
0,14.84


## Synthesize the question and results obtained

In [16]:
from genai.prompts import sythesizer_prompt

final_prompt = sythesizer_prompt(filtered_structured_object, query, results)
final_response = smart_llm.invoke(final_prompt)

## Your answer:

In [17]:
print(final_response.content)

You have spent $14.84 on shopping in the last 10 months.

Filtered by category: Shops and merchant: Not specified.
